In [56]:
import pickle
import cv2
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity

### Loading the moddel

In [57]:
file_path = "/Volumes/This One Is Totally Infectious/PROJEKTS/FACE ATTENDANCE/Face Recognition using Caffe/code/embeddings.pkl"

def load_embeddings(file_path):
    with open(file_path, 'rb') as file:
        embeddings, labels = pickle.load(file)
    return embeddings, labels

### Defining the main.py functions again: 
1. detect_faces()
2. extract_features()

In [58]:
def detect_faces(image, detector):
    h, w = image.shape[:2]
    blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), (104.0, 177.0, 123.0), False, False)
    detector.setInput(blob)
    detections = detector.forward()
    faces = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype(int)
            startX, startY, endX, endY = max(0, startX), max(0, startY), min(w-1, endX), min(h-1, endY)
            if endX > startX and endY > startY:
                faces.append((startX, startY, endX, endY))
    return faces

In [59]:
def extract_features(face_image, recognizer):
    if face_image.size == 0:
        return None
    blob = cv2.dnn.blobFromImage(face_image, 1.0 / 255, (96, 96), (0, 0, 0), True, False)
    recognizer.setInput(blob)
    return recognizer.forward()

### Face recognition using COSINE SIMILARITY

In [60]:
def recognize_face(input_embedding, embeddings, labels, threshold=0.5):
    max_similarity = 0
    recognized_label = "Unknown"
    for i, existing_embedding in enumerate(embeddings):
        similarity = cosine_similarity(input_embedding.reshape(1, -1), existing_embedding.reshape(1, -1))
        if similarity > max_similarity:
            max_similarity = similarity
            if max_similarity > threshold:
                recognized_label = labels[i]
    return recognized_label

### Prediction function

In [61]:
def predict_name(image_path, embeddings_file, face_detector, face_recognizer):

    embeddings, labels = load_embeddings(embeddings_file)                       #returns two list objs containing...  
    
    image = cv2.imread(image_path)
    if image is None:
        return "\nImage not found!\n"
    
    faces = detect_faces(image, face_detector)
    if not faces:
        return "\nNO FACES DETECTED!\n"
    
    for face in faces:
        x, y, x2, y2 = face
        face_img = image[y:y2, x:x2]

        if face_img.size != 0:
            input_embedding = extract_features(face_img, face_recognizer)

            if input_embedding is not None:
                input_embedding = input_embedding.flatten().reshape(1, -1)
                
                similarities = cosine_similarity(input_embedding, np.array(embeddings))
                max_index = np.argmax(similarities)

                return labels[max_index]
            
            else:
                return "\nUNABLE TO EXTRACT THE FEATURES FROM THE IMG\n"
            
        else:
            return "\nDETECTED FACE IS INVALID!\n"


In [67]:
face_detector = cv2.dnn.readNetFromCaffe(
    '/Volumes/This One Is Totally Infectious/PROJEKTS/FACE ATTENDANCE/Face Recognition using Caffe/models/deploy.prototxt',
    '/Volumes/This One Is Totally Infectious/PROJEKTS/FACE ATTENDANCE/Face Recognition using Caffe/models/res10_300x300_ssd_iter_140000.caffemodel'
)
face_recognizer = cv2.dnn.readNetFromTorch(
    '/Volumes/This One Is Totally Infectious/PROJEKTS/FACE ATTENDANCE/Face Recognition using Caffe/models/openface.nn4.small2.v1.t7'
)
image_path = '/Volumes/This One Is Totally Infectious/PROJEKTS/FACE ATTENDANCE/Face Recognition using Caffe/test_image/6.jpg'
embeddings_file = '/Volumes/This One Is Totally Infectious/PROJEKTS/FACE ATTENDANCE/Face Recognition using Caffe/code/embeddings.pkl'

predicted_name = predict_name(image_path, embeddings_file, face_detector, face_recognizer)
print(f"Predicted Name: {predicted_name}")

Predicted Name: Mayank


In [10]:
with open('/Volumes/This One Is Totally Infectious/PROJEKTS/FACE ATTENDANCE/Face Recognition using Caffe/code/embeddings.pkl', 'rb') as file:
    data = pickle.load(file)

print(type(data))  
print(data)        

<class 'tuple'>
([array([ 0.10390541,  0.08973685,  0.09227011, -0.1117952 ,  0.01405335,
        0.12820093,  0.05529164,  0.18148768, -0.10620878, -0.00118086,
       -0.03854811, -0.045233  ,  0.04429522, -0.09306667,  0.09486322,
       -0.02007025, -0.01603864, -0.11142244, -0.13113071,  0.00980295,
        0.08215465,  0.0520358 , -0.03676276,  0.02745063,  0.01531024,
        0.02854302, -0.04254608, -0.08297811,  0.04776426, -0.05422566,
        0.02600362, -0.02613063, -0.13234407,  0.11783869, -0.05434356,
       -0.063908  , -0.07020026,  0.15289402, -0.02734981,  0.04887344,
       -0.00542599, -0.03795717, -0.06726351,  0.09220842,  0.03906121,
        0.08057762,  0.02449566, -0.0323722 , -0.17972325,  0.03725804,
        0.10824316,  0.06516806, -0.1225169 , -0.03256536,  0.02833573,
       -0.05280614, -0.07224128,  0.11053221,  0.06747823, -0.13935037,
       -0.00355683, -0.07763845,  0.01321558, -0.01793155, -0.01296783,
       -0.12921019,  0.03160809,  0.11671867, 